In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from citipy import citipy
import time
from datetime import datetime
from config import weather_api_key


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
print(coordinates[0:3])

[(58.98324258578549, -116.35792343590958), (-39.77853393884714, -150.57694981619397), (71.22249276317251, 168.80700306053842)]


In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

774

In [5]:
 cities[0:5]

['high level', 'mataura', 'pevek', 'tazovskiy', 'morant bay']

In [6]:
# create endpoint url API looping
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=a2ed7e6720636c478addc422331e34d3


In [7]:
r = requests.get(url + "&q=" + cities[13].replace(" ","+") ).json()
r

{'coord': {'lon': 147.3294, 'lat': -42.8794},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 67.8,
  'feels_like': 66.58,
  'temp_min': 64.26,
  'temp_max': 69.26,
  'pressure': 1021,
  'humidity': 49},
 'visibility': 10000,
 'wind': {'speed': 1.99, 'deg': 135, 'gust': 4},
 'clouds': {'all': 75},
 'dt': 1643938299,
 'sys': {'type': 2,
  'id': 2002195,
  'country': 'AU',
  'sunrise': 1643915780,
  'sunset': 1643967164},
 'timezone': 39600,
 'id': 2163355,
 'name': 'Hobart',
 'cod': 200}

In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [ ]:
# Create an empty list to hold the weather data.
city_data = []
skip_count = 0
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        skip_count += 1
        continue
# print api summary notes.
print('{} cities skipped.'.format(skip_count)) 
print('data for {} cities acquired '.format(len(city_data)))

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
# Export the City_Data into a CSV.
city_data_df.to_csv('WeatherPy_Database.csv',index_label='City_ID')